In [1]:
%load_ext autoreload
%autoreload 2

# tool_calling with ReAct architecture can exhaust the token limit quickly.

I think I can solve this issue:  
- instead of let lm take all tool spaces, we can have a selection mechanism
- selection mechanism can be done many ways including cosine similarity (fast but prone to error), clustering (I think this one is better in grouping similar tool togther but it requires iterative training, so feasible or not we must try)
- can it be done by fine-tuning? If so what's pros and cons?
- is there any more way to do so?

In [2]:
# Cell 1: Setup
import dspy
from package.base import DriverLM, ModelResponse, Usage
import httpx
from typing import Dict

ollama_client = httpx.Client(timeout=600.0)

def ollama_request_fn(prompt=None, messages=None, temperature=0.0, max_tokens=512):
    if messages is None:
        messages = [{"role": "user", "content": prompt}]
    
    response = ollama_client.post(
        'http://localhost:11434/api/chat',
        json={
            "model": "llama3.2-vision:11b",
            "messages": messages,
            "stream": False,
            "options": {"temperature": temperature}
        }
    )
    response.raise_for_status()
    return response.json()

def ollama_output_fn(response: dict) -> ModelResponse:
    content = response.get("message", {}).get("content", "")
    model = response.get("model", "custom")
    
    usage = Usage(
        prompt_tokens=response.get("prompt_eval_count", 0),
        completion_tokens=response.get("eval_count", 0),
        total_tokens=response.get("prompt_eval_count", 0) + response.get("eval_count", 0)
    )
    
    return ModelResponse.from_text(text=content.strip(), usage=usage, model=model)

lm = DriverLM(
    request_fn=ollama_request_fn,
    output_fn=ollama_output_fn,
    cache=True
)
dspy.configure(lm=lm)


In [3]:
# Cell 2: Define Mock APIs
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    weather_db = {
        "Bangkok": "Sunny, 32°C, Humidity 70%",
        "Tokyo": "Cloudy, 18°C, Light rain expected",
        "London": "Rainy, 12°C, Windy",
        "New York": "Clear, 22°C, Pleasant"
    }
    return weather_db.get(city, f"Weather data not available for {city}")

def search_database(query: str) -> str:
    """Search product database."""
    products = {
        "laptop": "MacBook Pro 16-inch, $2499, In stock",
        "phone": "iPhone 15 Pro, $999, In stock",
        "tablet": "iPad Air, $599, Out of stock"
    }
    for key, value in products.items():
        if key in query.lower():
            return value
    return "No products found"

def calculate(expression: str) -> str:
    """Calculate mathematical expression."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

def get_user_info(user_id: int) -> Dict[str, str]:
    """Get user information by ID."""
    users = {
        1: {"name": "Alice", "email": "alice@example.com", "role": "admin"},
        2: {"name": "Bob", "email": "bob@example.com", "role": "user"},
        3: {"name": "Charlie", "email": "charlie@example.com", "role": "user"}
    }
    return users.get(user_id, {"error": "User not found"})


In [4]:
# Cell 3: Create ReAct Agent
from dspy import ReAct

class Question(dspy.Signature):
    """Answer questions using available tools."""
    question = dspy.InputField()
    answer = dspy.OutputField()

# Create agent with tools
agent = ReAct(Question, tools=[get_weather, search_database, calculate])


In [5]:
# Cell 4: Test Agent
# Test 1: Weather query
result = agent(question="What's the weather like in Bangkok?")
print("Q: What's the weather like in Bangkok?")
print(f"A: {result.answer}\n")

# Test 2: Product search
result = agent(question="Do you have laptops in stock?")
print("Q: Do you have laptops in stock?")
print(f"A: {result.answer}\n")

# Test 3: Calculation
result = agent(question="What is 25% of 1200?")
print("Q: What is 25% of 1200?")
print(f"A: {result.answer}\n")

# Test 4: Multi-step reasoning
result = agent(question="If a laptop costs $2499 and I have a 15% discount, how much will I pay?")
print("Q: If a laptop costs $2499 and I have a 15% discount, how much will I pay?")
print(f"A: {result.answer}")

Q: What's the weather like in Bangkok?
A: There are no products in the database that match the query for products suitable for sunny weather in Bangkok, and therefore, the total cost of such products is 0.

Q: Do you have laptops in stock?
A: Yes, we have laptops in stock, including the MacBook Pro 16-inch.

Q: What is 25% of 1200?
A: 300

Q: If a laptop costs $2499 and I have a 15% discount, how much will I pay?
A: $2124.15


In [6]:
len(lm.history)

24

In [7]:
print(lm.history[0]['messages'][0]['content'])

Your input fields are:
1. `question` (str): 
2. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['get_weather', 'search_database', 'calculate', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: get_weather; search_database; calculate; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer questions using available tools.
        
        You are an Agent. In each episode, you will be given the fields `q

In [8]:
print(lm.history[0]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]


Respond with the corresponding output fields, starting with the field `[[ ## next_thought ## ]]`, then `[[ ## next_tool_name ## ]]` (must be formatted as a valid Python Literal['get_weather', 'search_database', 'calculate', 'finish']), then `[[ ## next_tool_args ## ]]` (must be formatted as a valid Python dict[str, Any]), and then ending with the marker for `[[ ## completed ## ]]`.


In [9]:
print(lm.history[1]['messages'][0]['content'])

Your input fields are:
1. `question` (str): 
2. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['get_weather', 'search_database', 'calculate', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: get_weather; search_database; calculate; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Answer questions using available tools.
        
        You are an Agent. In each episode, you will be given the fields `q

In [10]:
print(lm.history[1]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

Respond with the corresponding output fields, starting with the field `[[ ## next_thought ## ]]`, then `[[ ## next_tool_name ## ]]` (must be formatted as a valid Python Literal['get_weather', 'search_database', 'calculate', 'finish']), then `[[ ## next_tool_args ## ]]` (must be formatted as a valid Python dict[str, Any]), and then ending with the marker for `[[ ## completed ## ]]`.


In [11]:
print(lm.history[2]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

[[ ## thought_1 ## ]]
Now that I have the current weather in Bangkok, I need to search the product database for products that are suitable for the weather.

[[ ## tool_name_1 ## ]]
search_database

[[ ## tool_args_1 ## ]]
{"query": "products suitable for sunny weather in Bangkok"}

[[ ## observation_1 ## ]]
No products found

Respond with the corresponding output fields, starting with the field `[[ ## next_thought ## ]]`, then `[[ ## next_tool_name ## ]]` (must be formatted as a valid Python Literal['get_weather', 'search_database', 'calculate', 'finish']), then `[[ ## next_tool_args ## ]]` (must be formatted as a valid Python dict[str, Any]), and then ending with the marker for `[[ 

In [13]:
print(lm.history[3]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

[[ ## thought_1 ## ]]
Now that I have the current weather in Bangkok, I need to search the product database for products that are suitable for the weather.

[[ ## tool_name_1 ## ]]
search_database

[[ ## tool_args_1 ## ]]
{"query": "products suitable for sunny weather in Bangkok"}

[[ ## observation_1 ## ]]
No products found

[[ ## thought_2 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangkok.

[[ ## tool_name_2 ## ]]
calculate

[[ ## tool_args_2 ## ]]
{"expression": "0"}

[[ ## observation_2 ## ]]
Result: 0

Respond with the corresponding output fields, starting with the field `[[ ## next_thought ## ]]`, then `[[ ## next_tool_n

In [14]:
print(lm.history[4]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

[[ ## thought_1 ## ]]
Now that I have the current weather in Bangkok, I need to search the product database for products that are suitable for the weather.

[[ ## tool_name_1 ## ]]
search_database

[[ ## tool_args_1 ## ]]
{"query": "products suitable for sunny weather in Bangkok"}

[[ ## observation_1 ## ]]
No products found

[[ ## thought_2 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangkok.

[[ ## tool_name_2 ## ]]
calculate

[[ ## tool_args_2 ## ]]
{"expression": "0"}

[[ ## observation_2 ## ]]
Result: 0

[[ ## thought_3 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangk

In [15]:
print(lm.history[5]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

[[ ## thought_1 ## ]]
Now that I have the current weather in Bangkok, I need to search the product database for products that are suitable for the weather.

[[ ## tool_name_1 ## ]]
search_database

[[ ## tool_args_1 ## ]]
{"query": "products suitable for sunny weather in Bangkok"}

[[ ## observation_1 ## ]]
No products found

[[ ## thought_2 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangkok.

[[ ## tool_name_2 ## ]]
calculate

[[ ## tool_args_2 ## ]]
{"expression": "0"}

[[ ## observation_2 ## ]]
Result: 0

[[ ## thought_3 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangk

In [18]:
print(lm.history[6]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

[[ ## thought_1 ## ]]
Now that I have the current weather in Bangkok, I need to search the product database for products that are suitable for the weather.

[[ ## tool_name_1 ## ]]
search_database

[[ ## tool_args_1 ## ]]
{"query": "products suitable for sunny weather in Bangkok"}

[[ ## observation_1 ## ]]
No products found

[[ ## thought_2 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangkok.

[[ ## tool_name_2 ## ]]
calculate

[[ ## tool_args_2 ## ]]
{"expression": "0"}

[[ ## observation_2 ## ]]
Result: 0

[[ ## thought_3 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangk

In [19]:
print(lm.history[7]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

[[ ## thought_1 ## ]]
Now that I have the current weather in Bangkok, I need to search the product database for products that are suitable for the weather.

[[ ## tool_name_1 ## ]]
search_database

[[ ## tool_args_1 ## ]]
{"query": "products suitable for sunny weather in Bangkok"}

[[ ## observation_1 ## ]]
No products found

[[ ## thought_2 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangkok.

[[ ## tool_name_2 ## ]]
calculate

[[ ## tool_args_2 ## ]]
{"expression": "0"}

[[ ## observation_2 ## ]]
Result: 0

[[ ## thought_3 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangk

In [20]:

print(lm.history[8]['messages'][1]['content'])

[[ ## question ## ]]
What's the weather like in Bangkok?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
I need to get the current weather in Bangkok to answer the question.

[[ ## tool_name_0 ## ]]
get_weather

[[ ## tool_args_0 ## ]]
{"city": "Bangkok"}

[[ ## observation_0 ## ]]
Sunny, 32°C, Humidity 70%

[[ ## thought_1 ## ]]
Now that I have the current weather in Bangkok, I need to search the product database for products that are suitable for the weather.

[[ ## tool_name_1 ## ]]
search_database

[[ ## tool_args_1 ## ]]
{"query": "products suitable for sunny weather in Bangkok"}

[[ ## observation_1 ## ]]
No products found

[[ ## thought_2 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangkok.

[[ ## tool_name_2 ## ]]
calculate

[[ ## tool_args_2 ## ]]
{"expression": "0"}

[[ ## observation_2 ## ]]
Result: 0

[[ ## thought_3 ## ]]
I need to calculate the total cost of the products that are suitable for the sunny weather in Bangk

In [34]:
print(lm.history[8]['response'].choices[0].message.content)

[[ ## reasoning ## ]]
The reasoning process for answering the question "What's the weather like in Bangkok?" involves several steps. Initially, I use the `get_weather` tool to retrieve the current weather in Bangkok, which is reported to be sunny with a temperature of 32°C and humidity of 70%. However, upon searching the product database for products suitable for sunny weather in Bangkok using the `search_database` tool, no products are found. This indicates that there are no products in the database that match the query. Consequently, the total cost of the products suitable for the weather in Bangkok is calculated using the `calculate` tool, but the result is consistently 0, suggesting that there are no products to calculate the cost for. Despite this, the process concludes with the `finish` tool, indicating that the task is complete.

[[ ## answer ## ]]
There are no products in the database that match the query for products suitable for sunny weather in Bangkok, and therefore, the to

In [ ]:
"""

<|start|> <|end|>

[[ ## field ## ]]

"""